In [27]:
import pandas as pd
import statsmodels.api as sm
from utils.ff_functions import create_coef_table
from utils.ff_functions import summarise_table
from utils.ff_functions import run_GRS

# Filtering Dates
In the paper they used the date range = [July 1963 - December 1991]

In [28]:
PATH = '../data/processed'

# Loading processed dfs
ff5 = pd.read_parquet(f'{PATH}/ff5_factors_monthly.parquet')
port = pd.read_parquet(f'{PATH}/ff_portfolios_25_monthly.parquet')

# Filtering FF5 columns to convert to FF3
ff3 = ff5[['Date', 'Mkt-RF', 'SMB', 'HML', 'RF']].copy()

# Filtering date to year range
ff3 = ff3[ff3['Date'].between('1963-07-01', '1991-12-31')]
port = port[port['Date'].between('1963-07-01', '1991-12-31')]

# Robustness check
assert len(ff3) == len(port), 'Observation sizes do not match'

# Creating list from columns except Date
ff3_cols = [c for c in ff3.columns if c != 'Date']
port_cols = [c for c in port.columns if c != 'Date']

# Applying numeric transformation
ff3[ff3_cols] = ff3[ff3_cols].apply(pd.to_numeric, errors='coerce')
port[port_cols] = port[port_cols].apply(pd.to_numeric, errors='coerce')

# Data Exploration

In [29]:
# Creating copies for descriptive stats
ff3_stat = ff3.copy()
port_stat = port.copy()

# Setting Date as index
ff3_stat.set_index('Date', inplace=True)
port_stat.set_index('Date', inplace=True)

In [30]:
ff3_stat.describe()

,Mkt-RF,SMB,HML,RF
count,342.000000,342.000000,342.000000,342.000000
mean,0.408246,0.302661,0.392105,0.552982
std,4.592660,2.962978,2.560401,0.218826
min,-23.240000,-10.080000,-9.720000,0.250000
25%,-2.185000,-1.370000,-1.070000,0.400000
50%,0.555000,0.115000,0.485000,0.510000
75%,3.535000,2.015000,1.680000,0.667500
max,16.100000,12.800000,8.600000,1.350000


In [31]:
port_stat.describe()

,SMALL LoBM,ME1 BM2,ME1 BM3,ME1 BM4,SMALL HiBM,ME2 BM1,ME2 BM2,ME2 BM3,ME2 BM4,ME2 BM5,...,ME4 BM1,ME4 BM2,ME4 BM3,ME4 BM4,ME4 BM5,BIG LoBM,ME5 BM2,ME5 BM3,ME5 BM4,BIG HiBM
count,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,...,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000,342.000000
mean,0.866010,1.218971,1.291350,1.420761,1.542843,0.946676,1.197346,1.394027,1.451832,1.535109,...,1.012612,0.918682,1.169203,1.345892,1.439576,0.922617,0.890408,0.908800,1.060339,1.081593
std,7.749803,6.779863,6.272839,5.912414,6.266145,7.272491,6.284909,5.723932,5.334407,6.015140,...,5.881375,5.401291,5.074356,4.915830,5.636360,4.894273,4.711281,4.374565,4.302269,4.779814
min,-34.165300,-30.677900,-28.593000,-28.358300,-28.484300,-32.484900,-31.941300,-28.660100,-24.661600,-29.254100,...,-25.547100,-28.685800,-24.024500,-18.499900,-23.468000,-21.651300,-22.449900,-22.028400,-15.446700,-18.701400
25%,-3.449450,-2.553900,-2.159625,-1.552625,-1.453300,-3.526050,-2.335675,-1.710825,-1.483550,-1.765050,...,-2.637550,-2.133400,-1.978525,-1.655525,-1.775150,-1.786050,-1.685675,-1.682925,-1.395350,-1.832850
50%,1.047550,1.176800,1.293900,1.425550,1.509300,1.071750,1.528050,1.552750,1.599300,1.686750,...,0.936850,1.060950,1.333650,1.497000,1.552650,0.989800,0.794550,0.996500,0.882300,1.052000
75%,5.431025,5.570000,5.077175,4.447575,4.803775,5.481350,5.089625,4.828525,4.843350,5.020800,...,4.843225,4.096025,4.420875,4.227225,4.995800,3.713625,3.798775,3.302600,3.533275,3.960300
max,25.198500,30.069000,28.281400,28.265600,33.203500,21.462600,26.620600,26.615300,27.180800,29.654200,...,21.209900,20.246000,23.887300,24.069700,27.854100,22.203100,16.491100,18.537400,19.866200,15.679700


# Time-Series FF3 regressions
For each of the 25 portfolios the regression equation is:
$$
\begin{equation}
R_{i,t} - R_{f,t}
=
\alpha_i
+
\beta_{i,M}\left(R_{M,t} - R_{f,t}\right)
+
\beta_{i,S}\,\mathrm{SMB}_t
+
\beta_{i,H}\,\mathrm{HML}_t
+
\varepsilon_{i,t}
\end{equation}
$$

Where:

$$
\begin{aligned}
R_{i,t} &:\ \text{return on portfolio } i \text{ at time } t \\
R_{f,t} &:\ \text{risk-free rate} \\
R_{M,t} - R_{f,t} &:\ \text{market excess return (MKT--RF)} \\
\mathrm{SMB}_t &:\ \text{size factor (Small Minus Big)} \\
\mathrm{HML}_t &:\ \text{value factor (High Minus Low)} \\
\alpha_i &:\ \text{pricing error (abnormal return)} \\
\varepsilon_{i,t} &:\ \text{regression residual}
\end{aligned}
$$

In [32]:
# Merging port and ff3 to regression df
df = port.merge(ff3[["Date", "Mkt-RF", "SMB", "HML", "RF"]], on="Date", how="inner")

# Dropping date from columns
portfolio_cols = port.columns.drop('Date')

# Converting to excess returns
df[portfolio_cols] = df[portfolio_cols].sub(df['RF'], axis=0)

# Creating results dictionary
ff3_results = {}

# Factors that will be used in regression
facts = ['Mkt-RF', 'SMB', 'HML']

for portfolio in portfolio_cols:
    Y = df[portfolio]
    X = sm.add_constant(df[facts])

    # Using heteroskedasticity robust SEs
    model = sm.OLS(Y,X).fit(cov_type='HAC', cov_kwds={'maxlags':12})
    ff3_results[portfolio] = model

In [33]:
# Creating table of coefficients for each portfolio
ff3_table = create_coef_table(ff3_results, factors=facts)
display(ff3_table.head())

# Summarising table
ff3_summary = summarise_table(ff3_table)
display(ff3_summary)

,Portfolio,alpha,t_alpha,p_alpha,R2,betaMkt-RF,betaSMB,betaHML
0,SMALL LoBM,-0.369237,-3.052528,0.0023,0.941882,1.033850,1.371117,-0.394751
1,ME1 BM2,-0.097297,-1.477217,0.1396,0.960748,0.965439,1.240546,-0.016106
2,ME1 BM3,-0.057833,-0.778675,0.4362,0.965137,0.944443,1.118101,0.184213
3,ME1 BM4,0.056688,0.924228,0.3554,0.967618,0.899777,1.067029,0.308115
4,SMALL HiBM,0.050713,0.771852,0.4402,0.961708,0.954114,1.144029,0.518693


mean[|alpha|)           0.092776
% sig alpha (<0.05)    16.000000
mean(R2)                0.934559
n_portfolios           25.000000
dtype: float64

In [34]:
ff3_table.to_csv('../results/ff3_results.csv', index=False)

# CAPM
We estimate CAPM to compare

In [35]:
# Creating results dictionary
capm_results = {}

for portfolio in portfolio_cols:
    Y = df[portfolio]
    X = sm.add_constant(df['Mkt-RF']) # only CAPM factor
    # Using heteroskedasticity robust SEs
    CAPM_model = sm.OLS(Y,X).fit(cov_type='HAC', cov_kwds={'maxlags':12})

    capm_results[portfolio] = CAPM_model

In [36]:
# Creating coefficient table
capm_table = create_coef_table(capm_results, factors=['Mkt-RF'])
display(capm_table.head())

# Summarising table
capm_summary = summarise_table(capm_table)
display(capm_summary)

,Portfolio,alpha,t_alpha,p_alpha,R2,betaMkt-RF
0,SMALL LoBM,-0.265484,-0.800503,0.4234,0.699712,1.417067
1,ME1 BM2,0.158852,0.624859,0.5321,0.703454,1.242234
2,ME1 BM3,0.267591,1.139517,0.2545,0.707923,1.153172
3,ME1 BM4,0.430205,2.002894,0.0452,0.687897,1.071841
4,SMALL HiBM,0.540799,2.194483,0.0282,0.644641,1.099978


mean[|alpha|)           0.257850
% sig alpha (<0.05)    36.000000
mean(R2)                0.794427
n_portfolios           25.000000
dtype: float64

In [37]:
capm_table.to_csv('../results/capm_results.csv', index=False)

# CAPM vs FF3 Comparison

In [38]:
# Setting index for alignment
capm_alpha = capm_table.set_index("Portfolio")[["alpha"]]
ff3_alpha  = ff3_table.set_index("Portfolio")[["alpha"]]

alpha_compare = capm_alpha.compare(ff3_alpha)
alpha_compare.columns = ['alpha_CAPM', 'alpha_FF3']

# Finding change in alpha
alpha_compare['alpha_diff'] = (
    alpha_compare['alpha_CAPM'] - alpha_compare['alpha_FF3']
)
display(alpha_compare)

,alpha_CAPM,alpha_FF3,alpha_diff
Portfolio,,,
SMALL LoBM,-0.265484,-0.369237,0.103753
ME1 BM2,0.158852,-0.097297,0.256149
ME1 BM3,0.267591,-0.057833,0.325424
ME1 BM4,0.430205,0.056688,0.373517
SMALL HiBM,0.540799,0.050713,0.490086
ME2 BM1,-0.188727,-0.136321,-0.052406
ME2 BM2,0.142000,-0.032283,0.174283
ME2 BM3,0.387247,0.131131,0.256116
ME2 BM4,0.477902,0.137620,0.340283


# Gibbons–Ross–Shanken (1989) Test

### GRS Test on FF3

In [39]:
# Creating df for grs
df_grs = port.merge(ff3[["Date","Mkt-RF","SMB","HML","RF"]], on="Date", how="inner")
portfolio_cols = port.columns.drop("Date")

# Subtracting rf
df_grs[portfolio_cols] = df_grs[portfolio_cols].sub(df_grs["RF"], axis=0)

In [40]:
facts = ['Mkt-RF', 'SMB', 'HML']

F_stat, pVal, _, _ = run_GRS(df_grs, portfolio_cols, facts)

print("GRS F:", F_stat)
print("p-value:", pVal)

GRS F: [[1.43505949]]
p-value: [[0.08473135]]


FF3 fails to reject at 5%, rejects at 10%. The alphas are jointly insignificant, meaning it prices all test portfolion on average.

### GRS Test on CAPM

In [41]:
capm_factors = ["Mkt-RF"]

F_capm, p_capm, _, _ = run_GRS(df_grs, portfolio_cols, capm_factors)

print("CAPM GRS F:", F_capm)
print("CAPM p-value:", p_capm)

CAPM GRS F: [[2.02888155]]
CAPM p-value: [[0.0030554]]


CAPM rejects strongly. Since we reject the null at least some alphas are systematically non-zero, meaning it can leave unpriced average returns.